In [ ]:
import requests
import re
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from io import StringIO
from datetime import date
from sklearn.ensemble import IsolationForest

# Define the customer key and sensor IDs
customer_key = "your_customer_key"  # Replace with your actual customer key
sensor_ids = "1,2,3"  # Replace with your actual sensor IDs

# Get the current year
today = date.today()
current_year = int(today.strftime("%Y"))
start_year = 2000
end_year = 2025

df_all_years = pd.DataFrame()

for year in range(start_year, current_year + 1):
    try:
        # Construct the URL with the customer key and sensor IDs
        URL = f"http://api.exactraq.net/v1/customers/{customer_key}/sensor_values.format?sensor_ids={sensor_ids}"

        # Send the GET request to the API
        rr = requests.get(URL)
        rr.raise_for_status()  # Check for HTTP errors

        # Assuming the response contains CSV data
        df = pd.read_csv(StringIO(rr.text))

        if df.empty:
            continue

        # Assuming the data has columns such as 'year', 'month', 'day', 'DISCHARGE'
        df["DATE"] = pd.to_datetime(df[['year', 'month', 'day']])
        df.drop(columns=["year", "month", "day"], inplace=True)
        df_all_years = pd.concat([df_all_years, df], ignore_index=True)

    except Exception as e:
        print(f"Error processing year {year}: {e}")

# Ensure we have data
if df_all_years.empty:
    print("No data collected. Exiting.")
    exit()